**1.Data Import and Library Integration**

In [ ]:
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import gym
import tensorflow as tf
from sklearn.decomposition import PCA
import xgboost as xgb
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, load_model
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import Huber
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.visualization.scatter import Scatter
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PolynomialMutation
import plotly.graph_objects as go

In [ ]:
# upload dataset
file_path = './matrix_f1_f2_200,000.csv'
data = pd.read_csv(file_path)
print(data.head())

**2.Model Training, Evaluation, and Optimization Workflow**

In [ ]:
### Model1 for y1 ###
# Prepare the data for Model1 (y1)
X = data.drop(columns=['y1', 'y2'])
y1 = data['y1']

# Split the data into training and testing sets for y1
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

# Create a pipeline with StandardScaler and Lasso using the specified alpha
pipeline1 = make_pipeline(StandardScaler(), Lasso(alpha=0.0001, max_iter=10000))

# Fit the model on the training data
pipeline1.fit(X_train, y1_train)
model1 = pipeline1

# Predict on the test set for y1
y1_pred = model1.predict(X_test)

# Calculate Mean Squared Error (MSE) and R² on the test set for y1
mse_y1 = mean_squared_error(y1_test, y1_pred)
r2_y1 = r2_score(y1_test, y1_pred)

print("Model1 - y1:")
print("Alpha used:", 0.0001)
print("Mean Squared Error (MSE) on Test Set:", mse_y1)
print("R² Score on Test Set:", r2_y1)

In [ ]:
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred,
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot for Model1 (y1)
plot_actual_vs_predicted(y1_test, y1_pred, "Actual vs Predicted y1 Values (Model1)", "Actual y1", "Predicted y1")

In [ ]:
X = data.drop(columns=['y1', 'y2'])
y2 = data['y2']

# Split the data into training and testing sets for y2
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

# Create a pipeline with StandardScaler and Lasso using the specified alpha
pipeline2 = make_pipeline(StandardScaler(), Lasso(alpha=0.0001, max_iter=50000))

# Fit the model on the training data
pipeline2.fit(X_train, y2_train)
model2 = pipeline2

# Predict on the test set for y2
y2_pred = model2.predict(X_test)

# Calculate Mean Squared Error (MSE) and R² on the test set for y2
mse_y2 = mean_squared_error(y2_test, y2_pred)
r2_y2 = r2_score(y2_test, y2_pred)

print("Model2 - y2:")
print("Alpha used:", 0.0001)
print("Mean Squared Error (MSE) on Test Set:", mse_y2)
print("R² Score on Test Set:", r2_y2)

In [ ]:
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred,
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot for Model2 (y2)
plot_actual_vs_predicted(y2_test, y2_pred, "Actual vs Predicted y2 Values (Model2)", "Actual y2", "Predicted y2")

In [ ]:
#XGBoost Model3
X = data.drop(columns=['y1', 'y2'])
y1 = data['y1']

# Split the data into training and testing sets for y1
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

# Define the XGBoost regressor with L1 regularization (alpha) for model3
model3 = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.5,  # L1 regularization parameter
    random_state=42
)

# Fit the model on the training data for y1
model3.fit(X_train, y1_train)

# Evaluate on the test set for y1
y1_pred = model3.predict(X_test)
test_mse_y1 = mean_squared_error(y1_test, y1_pred)
r2_y1 = r2_score(y1_test, y1_pred)

print("Test MSE:", test_mse_y1)

In [ ]:
# Extract the initial values from row 14 for columns X1-X50
initial_values = data.iloc[13, 2:52].values  # Extracting X1-X50 from row 14

# Define the optimization problem
class F1F2Problem(Problem):
    def __init__(self, model_y1, model_y2, initial_values):
        n_var = 70  # Total number of features
        xl_x1_x50 = initial_values  # Fix X1-X50
        xu_x1_x50 = initial_values  # Fix X1-X50

        # Define the range for X51-X70 (10 to 49)
        xl_x51_x70 = np.full(20, 10)
        xu_x51_x70 = np.full(20, 49)

        # Combine all the variable bounds
        xl = np.concatenate((xl_x1_x50, xl_x51_x70))
        xu = np.concatenate((xu_x1_x50, xu_x51_x70))

        super().__init__(n_var=n_var, n_obj=2, n_constr=0, xl=xl, xu=xu)
        self.model_y1 = model_y1
        self.model_y2 = model_y2

    def _evaluate(self, X, out, *args, **kwargs):
        # Scale the input data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        # Predict F1 and F2 using the trained Lasso models
        f1 = self.model_y1.predict(X_scaled)
        f2 = self.model_y2.predict(X_scaled)
        out["F"] = np.column_stack([f1, f2])

# Create the problem instance for model1 and model2
problem = F1F2Problem(model_y1=model1, model_y2=model2, initial_values=initial_values)

# Define the algorithm with updated parameters
algorithm = NSGA2(
    pop_size=100,  # Population size
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.95, eta=10),  # Crossover probability and eta
    mutation=PolynomialMutation(prob=0.25, eta=25)  # Mutation rate
)

# Set up colors for the different runs
colors = ['red', 'blue', 'green', 'orange', 'purple']

# Set a fixed seed for all runs to make them identical
fixed_seed = 42

# Create a figure for the combined plot
plt.figure(figsize=(10, 7))

# Loop to run the optimization 5 times and plot the Pareto fronts in different colors
for i in range(5):
    # Execute the optimization with the fixed seed
    res = minimize(problem, algorithm, ('n_gen', 100), seed=fixed_seed + i, save_history=True, verbose=True)

    # Plot the Pareto Frontier for this run in a different color
    plt.scatter(res.F[:, 0], res.F[:, 1], color=colors[i], label=f'Run {i+1}')

# Calculate the min and max values for f1 and f2 across all runs
f1_min, f1_max = res.F[:, 0].min(), res.F[:, 0].max()
f2_min, f2_max = res.F[:, 1].min(), res.F[:, 1].max()

# Add some padding for better visualization
f1_range = f1_max - f1_min
f2_range = f2_max - f2_min

plt.xlim(f1_min - 0.05 * f1_range, f1_max + 0.05 * f1_range)  # Set the range for f1 on the x-axis
plt.ylim(f2_min - 0.05 * f2_range, f2_max + 0.05 * f2_range)  # Set the range for f2 on the y-axis

# Add labels and legend
plt.xlabel('Objective 1 (f1)')
plt.ylabel('Objective 2 (f2)')
plt.title('Pareto Front for NSGA-II with Lasso Model')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Plot for Model3 (y1)
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred,
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot the predicted vs actual values for y1 (Model3)
plot_actual_vs_predicted(y1_test, y1_pred, "Actual vs Predicted y1 Values (Model3)", "Actual y1", "Predicted y1")

In [ ]:
X = data.drop(columns=['y1', 'y2'])
y2 = data['y2']

# Split the data into training and testing sets for y2
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

# Define the XGBoost regressor with L1 regularization (alpha) for model4
model4 = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=200,
    max_depth=7,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    alpha=0.5,  # L1 regularization parameter
    random_state=42
)

# Fit the model on the training data for y2
model4.fit(X_train, y2_train)

# Evaluate on the test set for y2
y2_pred = model4.predict(X_test)
test_mse_y2 = mean_squared_error(y2_test, y2_pred)
r2_y2 = r2_score(y2_test, y2_pred)

print("Test MSE:", test_mse_y2)

In [ ]:
# Plot for Model4 (y2)
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred,
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot the predicted vs actual values for y2 (Model4)
plot_actual_vs_predicted(y2_test, y2_pred, "Actual vs Predicted y2 Values (Model4)", "Actual y2", "Predicted y2")

In [ ]:
# Extract the initial values from row 14 for columns X1-X50
initial_values = data.iloc[13, 2:52].values  # Extracting X1-X50 from row 14

# Define the optimization problem
class F1F2Problem(Problem):
    def __init__(self, model_y1, model_y2, initial_values):
        n_var = 70  # Total number of features
        xl_x1_x50 = initial_values  # Fix X1-X50
        xu_x1_x50 = initial_values  # Fix X1-X50

        # Define the range for X51-X70 (10 to 49)
        xl_x51_x70 = np.full(20, 10)
        xu_x51_x70 = np.full(20, 49)

        # Combine all the variable bounds
        xl = np.concatenate((xl_x1_x50, xl_x51_x70))
        xu = np.concatenate((xu_x1_x50, xu_x51_x70))

        super().__init__(n_var=n_var, n_obj=2, n_constr=0, xl=xl, xu=xu)
        self.model_y1 = model_y1
        self.model_y2 = model_y2

    def _evaluate(self, X, out, *args, **kwargs):
        # Scale the input data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        # Predict F1 and F2 using the trained models
        f1 = self.model_y1.predict(X_scaled)
        f2 = self.model_y2.predict(X_scaled)
        out["F"] = np.column_stack([f1, f2])

# Create the problem instance for model3 and model4
problem = F1F2Problem(model_y1=model3, model_y2=model4, initial_values=initial_values)

# Define the algorithm with updated parameters
algorithm = NSGA2(
    pop_size=100,  # Population size
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.95, eta=10),  # Crossover probability and eta
    mutation=PolynomialMutation(prob=0.25, eta=25)  # Mutation rate
)

# Set up colors for the different runs
colors = ['red', 'blue', 'green', 'orange', 'purple']

# Set a fixed seed for all runs to make them identical
fixed_seed = 42

# Create a figure for the combined plot
plt.figure(figsize=(10, 7))

# Loop to run the optimization 5 times and plot the Pareto fronts in different colors
for i in range(5):
    # Execute the optimization with the fixed seed
    res = minimize(problem, algorithm, ('n_gen', 100), seed=fixed_seed + i, save_history=True, verbose=True)

    # Plot the Pareto Frontier for this run in a different color
    plt.scatter(res.F[:, 0], res.F[:, 1], color=colors[i], label=f'Run {i+1}')

# Calculate the min and max values for f1 and f2 across all runs
f1_min, f1_max = res.F[:, 0].min(), res.F[:, 0].max()
f2_min, f2_max = res.F[:, 1].min(), res.F[:, 1].max()

# Add some padding for better visualization
f1_range = f1_max - f1_min
f2_range = f2_max - f2_min

plt.xlim(f1_min - 0.05 * f1_range, f1_max + 0.05 * f1_range)  # Set the range for f1 on the x-axis
plt.ylim(f2_min - 0.05 * f2_range, f2_max + 0.05 * f2_range)  # Set the range for f2 on the y-axis

# Add labels and legend
plt.xlabel('Objective 1 (f1)')
plt.ylabel('Objective 2 (f2)')
plt.title('Pareto Front for NSGA-II with XGBoost Model')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Prepare the data for y1:64/32:0.019(PCA:0.022); 32:0.022; 128/64/32:0.031;
# MLP Prepare the data for y1
X = data.drop(columns=['y1', 'y2'])
y1 = data[['y1']]

# Split the data into training and testing sets for y1
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the MLP model for y1 (model5)
model5 = Sequential()
model5.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model5.add(Dropout(0.2))
model5.add(Dense(32, activation='relu'))
model5.add(Dense(1, activation='linear'))  # One output node for y1

# Compile the model
model5.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping to prevent overfitting
early_stopping_y1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model and measure time
start_time_model5 = time.time()
history5 = model5.fit(X_train_scaled, y1_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping_y1])
end_time_model5 = time.time()

# Evaluate the model
test_loss_model5 = model5.evaluate(X_test_scaled, y1_test)
print(f"Test MSE for y1 (model5): {test_loss_model5}")

# Predict on test data
y1_pred = model5.predict(X_test_scaled)

train_time_model5 = end_time_model5 - start_time_model5
print(f"Training time for y1 model (model5): {train_time_model5} seconds")

In [ ]:
import plotly.graph_objects as go

def plot_actual_vs_predicted_y1(y_true, y_pred, title):
    fig = go.Figure()

    # Actual vs. Predicted for y1
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true.flatten(),
            y=y_pred.flatten(),
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    fig.add_trace(
        go.Scatter(
            x=[0, 3],
            y=[0, 3],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="Actual y1",
        yaxis_title="Predicted y1",
        width=700,
        height=500,
        xaxis=dict(range=[0, 3]),
        yaxis=dict(range=[0, 3])
    )

    # Show plot
    fig.show()

# Plot for model5 (y1)
plot_actual_vs_predicted_y1(y1_test.values, y1_pred, "Actual vs Predicted y1 Values (Model5)")

In [ ]:
#5times or around to trade-off 100，000：6.29；200，000：3.99;500,000:5.01
#test set; unseen data (userbility)
# Prepare the data for y2
X = data.drop(columns=['y1', 'y2'])
y2 = data[['y2']]

# Split the data into training and testing sets for y2
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

# Standardize the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build the MLP model for y2 (model6)
model6 = Sequential()
model6.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model6.add(Dropout(0.2))
model6.add(Dense(32, activation='relu'))
model6.add(Dense(1, activation='linear'))  # One output node for y2

# Compile the model
model6.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping to prevent overfitting
early_stopping_y2 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model and measure time
start_time_model6 = time.time()
history6 = model6.fit(X_train_scaled, y2_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1, callbacks=[early_stopping_y2])
end_time_model6 = time.time()

# Evaluate the model
test_loss_model6 = model6.evaluate(X_test_scaled, y2_test)
print(f"Test MSE for y2 (model6): {test_loss_model6}")

# Predict on test data
y2_pred = model6.predict(X_test_scaled)

train_time_model6 = end_time_model6 - start_time_model6
print(f"Training time for y2 model (model6): {train_time_model6} seconds")

In [ ]:
def plot_actual_vs_predicted_y2(y_true, y_pred, title):
    fig = go.Figure()

    # Actual vs. Predicted for y2
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true.flatten(),
            y=y_pred.flatten(),
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],  # Adjusted x-axis range based on realistic range
            y=[min_value, max_value],  # Adjusted y-axis range based on realistic range
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title="Actual y2",
        yaxis_title="Predicted y2",
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),  # Adjusted x-axis range
        yaxis=dict(range=[min_value, max_value])   # Adjusted y-axis range
    )

    # Show plot
    fig.show()

# Plot for model6 (y2)
plot_actual_vs_predicted_y2(y2_test.values, y2_pred, "Actual vs Predicted y2 Values (Model6)")


In [ ]:
# Extract the initial values from row 14 for columns X1-X50
initial_values = data.iloc[13, 2:52].values  # Extracting X1-X50 from row 14

# Define the optimization problem
class F1F2Problem(Problem):
    def __init__(self, model_y1, model_y2, initial_values):
        n_var = 70  # Total number of features
        xl_x1_x50 = initial_values  # Fix X1-X50
        xu_x1_x50 = initial_values  # Fix X1-X50

        # Define the range for X51-X70 (10 to 49)
        xl_x51_x70 = np.full(20, 10)
        xu_x51_x70 = np.full(20, 49)

        # Combine all the variable bounds
        xl = np.concatenate((xl_x1_x50, xl_x51_x70))
        xu = np.concatenate((xu_x1_x50, xu_x51_x70))

        super().__init__(n_var=n_var, n_obj=2, n_constr=0, xl=xl, xu=xu)
        self.model_y1 = model_y1
        self.model_y2 = model_y2

    def _evaluate(self, X, out, *args, **kwargs):
        # Scale the input data
        X_scaled = scaler.transform(X)
        # Predict F1 and F2 using the trained MLP models
        f1 = self.model_y1.predict(X_scaled)
        f2 = self.model_y2.predict(X_scaled)
        out["F"] = np.column_stack([f1, f2])

# Create the problem instance
problem = F1F2Problem(model_y1=model5, model_y2=model6, initial_values=initial_values)

# Define the algorithm with updated parameters
algorithm = NSGA2(
    pop_size=100,  # Increased population size
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.95, eta=10),  # Increased crossover probability and decreased eta
    mutation=PolynomialMutation(prob=0.25, eta=25)  # Increased mutation rate
)

# Set up colors for the different runs
colors = ['red', 'blue', 'green', 'orange', 'purple']

# Set a fixed seed for all runs to make them identical
fixed_seed = 42

# Create a figure for the combined plot
plt.figure(figsize=(10, 7))

# Loop to run the optimization 5 times and plot the Pareto fronts in different colors
for i in range(5):
    # Execute the optimization with the fixed seed
    res = minimize(problem, algorithm, ('n_gen', 100), seed=fixed_seed+i, save_history=True, verbose=True)

    # Plot the Pareto Frontier for this run in a different color
    plt.scatter(res.F[:, 0], res.F[:, 1], color=colors[i], label=f'Run {i+1}')

# Calculate the min and max values for f1 and f2 across all runs
f1_min, f1_max = res.F[:, 0].min(), res.F[:, 0].max()
f2_min, f2_max = res.F[:, 1].min(), res.F[:, 1].max()

# Add some padding for better visualization
f1_range = f1_max - f1_min
f2_range = f2_max - f2_min

plt.xlim(f1_min - 0.05 * f1_range, f1_max + 0.05 * f1_range)  # Set the range for f1 on the x-axis
plt.ylim(f2_min - 0.05 * f2_range, f2_max + 0.05 * f2_range)  # Set the range for f2 on the y-axis

# Add labels and legend
plt.xlabel('Objective 1 (f1)')
plt.ylabel('Objective 2 (f2)')
plt.title('Pareto Front for NSGA-II with MLP Model')
plt.legend()

# Show the plot
plt.show()

In [ ]:
# Model7 for y1
# Prepare the data for model7 (y1)
X = data.drop(columns=['y1', 'y2']).values
y1 = data['y1'].values

# Reshape X to be 3D as required by LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into training and testing sets for y1
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Construct the LSTM model for y1 (model7)
model7 = Sequential()
model7.add(LSTM(64, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), activation='relu'))
model7.add(Dropout(0.2))
model7.add(Dense(1, activation='linear'))

# Compile the model
model7.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model and measure time
start_time_model7 = time.time()
history7 = model7.fit(X_train_scaled, y1_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)
end_time_model7 = time.time()

# Calculate training time
train_time_model7 = end_time_model7 - start_time_model7
print(f"Training time for y1 model (model7): {train_time_model7} seconds")

# Evaluate the model on the test set
test_loss_model7 = model7.evaluate(X_test_scaled, y1_test)
print(f"Test MSE for model7 (y1): {test_loss_model7}")

# Predict on the test set
y1_pred_model7 = model7.predict(X_test_scaled)

In [ ]:
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred.flatten(),  # Flattening y_pred to match y_true
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot the predicted vs actual values for y1 (Model7)
plot_actual_vs_predicted(y1_test, y1_pred, "Actual vs Predicted y1 Values (Model7)", "Actual y1", "Predicted y1")

In [ ]:
# Model8 for y2
# Prepare the data for model8 (y2)
X = data.drop(columns=['y1', 'y2']).values
y2 = data['y2'].values

# Reshape X to be 3D as required by LSTM (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into training and testing sets for y2
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.2, random_state=42)

# Standardize data
X_train_scaled = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_test_scaled = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

# Construct the LSTM model for y2 (model8)
model8 = Sequential()
model8.add(LSTM(64, input_shape=(X_train_scaled.shape[1], X_train_scaled.shape[2]), activation='relu'))
model8.add(Dropout(0.2))
model8.add(Dense(1, activation='linear'))

# Compile the model
model8.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train the model and measure time
start_time_model8 = time.time()
history8 = model8.fit(X_train_scaled, y2_train, validation_split=0.2, epochs=100, batch_size=32, verbose=1)
end_time_model8 = time.time()

# Calculate training time
train_time_model8 = end_time_model8 - start_time_model8
print(f"Training time for y2 model (model8): {train_time_model8} seconds")

# Evaluate the model on the test set
test_loss_model8 = model8.evaluate(X_test_scaled, y2_test)
print(f"Test MSE for model8 (y2): {test_loss_model8}")

# Predict on the test set
y2_pred_model8 = model8.predict(X_test_scaled)

In [ ]:
# Plot the predicted vs actual values for y2 (Model8)
def plot_actual_vs_predicted(y_true, y_pred, title, xaxis_title, yaxis_title):
    fig = go.Figure()

    # Actual vs. Predicted
    fig.add_trace(
        go.Scatter(
            mode='markers',
            x=y_true,
            y=y_pred.flatten(),  # Flattening y_pred to match y_true
            marker=dict(
                color='rgba(40, 40, 250, 0.3)',
                size=5,
            ),
            name='Actual vs Predicted'
        )
    )

    # Add a 45-degree line to show the perfect prediction
    min_value = min(y_true.min(), y_pred.min())
    max_value = max(y_true.max(), y_pred.max())

    fig.add_trace(
        go.Scatter(
            x=[min_value, max_value],
            y=[min_value, max_value],
            mode='lines',
            line=dict(color='red', dash='dash'),
            showlegend=False,
            name='Perfect Prediction Line'
        )
    )

    # Update layout
    fig.update_layout(
        title=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        width=700,
        height=500,
        xaxis=dict(range=[min_value, max_value]),
        yaxis=dict(range=[min_value, max_value])
    )

    # Show plot
    fig.show()

# Plot the predicted vs actual values for y2 (Model8)
plot_actual_vs_predicted(y2_test, y2_pred, "Actual vs Predicted y2 Values (Model8)", "Actual y2", "Predicted y2")

In [ ]:
# Extract the initial values from row 14 for columns X1-X50
initial_values = data.iloc[13, 2:52].values  # Extracting X1-X50 from row 14

# Define the optimization problem
class F1F2Problem(Problem):
    def __init__(self, model_y1, model_y2, initial_values):
        n_var = 70  # Total number of features
        xl_x1_x50 = initial_values  # Fix X1-X50
        xu_x1_x50 = initial_values  # Fix X1-X50

        # Define the range for X51-X70 (10 to 49)
        xl_x51_x70 = np.full(20, 10)
        xu_x51_x70 = np.full(20, 49)

        # Combine all the variable bounds
        xl = np.concatenate((xl_x1_x50, xl_x51_x70))
        xu = np.concatenate((xu_x1_x50, xu_x51_x70))

        super().__init__(n_var=n_var, n_obj=2, n_constr=0, xl=xl, xu=xu)
        self.model_y1 = model_y1
        self.model_y2 = model_y2

    def _evaluate(self, X, out, *args, **kwargs):
        # Reshape X to be 3D as required by LSTM (samples, timesteps, features)
        X_reshaped = X.reshape((X.shape[0], 1, X.shape[1]))

        # Scale the input data
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_reshaped.reshape(-1, X_reshaped.shape[-1])).reshape(X_reshaped.shape)

        # Predict F1 and F2 using the trained LSTM models
        f1 = self.model_y1.predict(X_scaled)
        f2 = self.model_y2.predict(X_scaled)

        # Filter out negative values by setting them to zero or a small positive value
        f1[f1 < 0] = 0
        f2[f2 < 0] = 0

        out["F"] = np.column_stack([f1, f2])

# Create the problem instance for model7 and model8
problem = F1F2Problem(model_y1=model7, model_y2=model8, initial_values=initial_values)

# Define the algorithm with updated parameters
algorithm = NSGA2(
    pop_size=100,  # Population size
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.95, eta=10),  # Crossover probability and eta
    mutation=PolynomialMutation(prob=0.25, eta=25)  # Mutation rate
)

# Set up colors for the different runs
colors = ['red', 'blue', 'green', 'orange', 'purple']

# Set a fixed seed for all runs to make them identical
fixed_seed = 42

# Create a figure for the combined plot
plt.figure(figsize=(10, 7))

# Loop to run the optimization 5 times and plot the Pareto fronts in different colors
for i in range(5):
    # Execute the optimization with the fixed seed
    res = minimize(problem, algorithm, ('n_gen', 200), seed=fixed_seed + i, save_history=True, verbose=True)

    # Plot the Pareto Frontier for this run in a different color
    plt.scatter(res.F[:, 0], res.F[:, 1], color=colors[i], label=f'Run {i+1}')

# Calculate the min and max values for f1 and f2 across all runs
f1_min, f1_max = res.F[:, 0].min(), res.F[:, 0].max()
f2_min, f2_max = res.F[:, 1].min(), res.F[:, 1].max()

# Add some padding for better visualization
f1_range = f1_max - f1_min
f2_range = f2_max - f2_min

plt.xlim(f1_min - 0.05 * f1_range, f1_max + 0.05 * f1_range)  # Set the range for f1 on the x-axis
plt.ylim(f2_min - 0.05 * f2_range, f2_max + 0.05 * f2_range)  # Set the range for f2 on the y-axis

# Add labels and legend
plt.xlabel('Objective 1 (f1)')
plt.ylabel('Objective 2 (f2)')
plt.title('Pareto Front for NSGA-II with LSTM Model')
plt.legend()
# Show the plot
plt.show()

In [ ]:
# Define the optimization problem
class F1F2Problem(Problem):
    def __init__(self, model_y1, model_y2, initial_values, is_lstm=False):
        n_var = 70  # Total number of features
        xl_x1_x50 = initial_values  # Fix X1-X50
        xu_x1_x50 = initial_values  # Fix X1-X50

        # Define the range for X51-X70 (10 to 49)
        xl_x51_x70 = np.full(20, 10)
        xu_x51_x70 = np.full(20, 49)

        # Combine all the variable bounds
        xl = np.concatenate((xl_x1_x50, xl_x51_x70))
        xu = np.concatenate((xu_x1_x50, xu_x51_x70))

        super().__init__(n_var=n_var, n_obj=2, n_constr=0, xl=xl, xu=xu)
        self.model_y1 = model_y1
        self.model_y2 = model_y2
        self.is_lstm = is_lstm

    def _evaluate(self, X, out, *args, **kwargs):
    # Scale the input data
      scaler = StandardScaler()
      X_scaled = scaler.fit_transform(X)

      if self.is_lstm:
          # Reshape X_scaled to be 3D if the model is LSTM
          X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

      # Predict F1 and F2 using the trained models
      f1 = self.model_y1.predict(X_scaled)
      f2 = self.model_y2.predict(X_scaled)

      # Filter out negative values by setting them to zero or a small positive value
      f1[f1 < 0] = 0
      f2[f2 < 0] = 0

      out["F"] = np.column_stack([f1, f2])





# Extract the initial values from row 14 for columns X1-X50
initial_values = data.iloc[13, 2:52].values  # Extracting X1-X50 from row 14

# Define the algorithm with updated parameters
algorithm = NSGA2(
    pop_size=100,  # Population size
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.95, eta=10),  # Crossover probability and eta
    mutation=PolynomialMutation(prob=0.25, eta=25)  # Mutation rate
)

# Set up markers and colors for the different models
markers = ['s', 'o', 'D', '^']  # Square, circle, diamond, triangle
colors = ['red', 'blue', 'green', 'orange']

# Create a figure for the combined plot
plt.figure(figsize=(10, 7))

# List of model combinations and LSTM flag
model_combinations = [
    (model1, model2, False),  # Lasso
    (model3, model4, False),  # XGBoost
    (model5, model6, False),  # MLP
    (model7, model8, True)    # LSTM
]

# Variables to track the min and max values for f1 and f2
f1_min, f1_max = np.inf, -np.inf
f2_min, f2_max = np.inf, -np.inf

# Loop to run the optimization for each model combination and plot the Pareto fronts
for idx, (model_y1, model_y2, is_lstm) in enumerate(model_combinations):
    # Instantiate the problem with the current model combination
    problem = F1F2Problem(model_y1=model_y1, model_y2=model_y2, initial_values=initial_values, is_lstm=is_lstm)

    # Execute the optimization with a fixed seed for reproducibility
    res = minimize(problem, algorithm, ('n_gen', 100), seed=42, save_history=True, verbose=True)

    # Update the min and max values for f1 and f2
    f1_min = min(f1_min, res.F[:, 0].min())
    f1_max = max(f1_max, res.F[:, 0].max())
    f2_min = min(f2_min, res.F[:, 1].min())
    f2_max = max(f2_max, res.F[:, 1].max())

    # Plot the Pareto Frontier for this model combination
    plt.scatter(res.F[:, 0], res.F[:, 1], color=colors[idx], marker=markers[idx], label=f'Model Set {idx+1}', s=100)

# Add some padding for better visualization
f1_range = f1_max - f1_min
f2_range = f2_max - f2_min

plt.xlim(f1_min - 0.05 * f1_range, f1_max + 0.05 * f1_range)  # Dynamically set the range for f1 on the x-axis
plt.ylim(f2_min - 0.05 * f2_range, f2_max + 0.05 * f2_range)  # Dynamically set the range for f2 on the y-axis

# Add labels, title, and legend
plt.xlabel('Objective 1 (f1)')
plt.ylabel('Objective 2 (f2)')
plt.title('Comparison of Pareto Fronts from Different Model Combinations')
plt.legend(labels=['Lasso (Model 1 & 2)', 'XGBoost (Model 3 & 4)', 'MLP (Model 5 & 6)', 'LSTM (Model 7 & 8)'])

# Show the plot
plt.show()

In [ ]:
# Convert training time from seconds to minutes
train_time_model5_minutes = train_time_model5 / 60
train_time_model7_minutes = train_time_model7 / 60

# Data for plotting
models_57 = ['Model 5', 'Model 7']
training_times_57 = [train_time_model5_minutes, train_time_model7_minutes]
test_losses_57 = [test_loss_model5, test_loss_model7]

# Set up the color palette
palette = sns.color_palette("husl", 2)

# Create the figure and axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the training time as a bar chart on the left y-axis
ax1.bar(models_57, training_times_57, color=palette[0], alpha=0.6, label='Training Time (min)')
ax1.set_xlabel('Models', fontsize=14)
ax1.set_ylabel('Training Time (min)', fontsize=14, color=palette[0])
ax1.tick_params(axis='y', labelcolor=palette[0])
ax1.set_ylim(0, max(training_times_57) * 1.1)  # Start y-axis from 0

# Create the second y-axis for test MSE
ax2 = ax1.twinx()
ax2.plot(models_57, test_losses_57, color=palette[1], marker='o', linestyle='-', linewidth=2, label='Test MSE')
ax2.set_ylabel('Test MSE', fontsize=14, color=palette[1])
ax2.tick_params(axis='y', labelcolor=palette[1])
ax2.set_ylim(0, max(test_losses_57) * 1.1)  # Start y-axis from 0

# Add a title and grid
plt.title('Comparison of Training Time and Test MSE (Model 5 vs Model 7)', fontsize=16)
ax1.grid(True, linestyle='--', alpha=0.7)

# Adjust layout to ensure everything fits without overlap
fig.tight_layout()

# Show the plot
plt.show()

In [ ]:
# Convert training time from seconds to minutes
train_time_model6_minutes = train_time_model6 / 60
train_time_model8_minutes = train_time_model8 / 60

# Data for plotting
models_68 = ['Model 6', 'Model 8']
training_times_68 = [train_time_model6_minutes, train_time_model8_minutes]
test_losses_68 = [test_loss_model6, test_loss_model8]

# Set up the color palette
palette = sns.color_palette("husl", 2)

# Create the figure and axis
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot the training time as a bar chart on the left y-axis
ax1.bar(models_68, training_times_68, color=palette[0], alpha=0.6, label='Training Time (min)')
ax1.set_xlabel('Models', fontsize=14)
ax1.set_ylabel('Training Time (min)', fontsize=14, color=palette[0])
ax1.tick_params(axis='y', labelcolor=palette[0])
ax1.set_ylim(0, max(training_times_68) * 1.1)  # Start y-axis from 0

# Create the second y-axis for test MSE
ax2 = ax1.twinx()
ax2.plot(models_68, test_losses_68, color=palette[1], marker='o', linestyle='-', linewidth=2, label='Test MSE')
ax2.set_ylabel('Test MSE', fontsize=14, color=palette[1])
ax2.tick_params(axis='y', labelcolor=palette[1])
ax2.set_ylim(0, max(test_losses_68) * 1.1)  # Start y-axis from 0

# Add a title and grid
plt.title('Comparison of Training Time and Test MSE (Model 6 vs Model 8)', fontsize=16)
ax1.grid(True, linestyle='--', alpha=0.7)

# Adjust layout to ensure everything fits without overlap
fig.tight_layout()

# Show the plot
plt.show()